# **<span style="color: #0098cd;">Procesamiento del Lenguaje Natural para la detección de mensajes de odio</span>**


## **Objetivos** 

## **Descripción**

### **Carga y preprocesamiento del texto**

In [65]:
#!pip install spacy
#!pip install chardet
#python -m spacy download es_core_news_md

import pathlib 
import spacy
import pandas as pd
from spacy import displacy
import csv
import chardet

#### **Carga del modelo y detección de codificación**

In [66]:
try:
    nlp = spacy.load("es_core_news_md")  # Cargo el modelo de Spacy para español
except OSError:
    print("El modelo 'es_core_news_md' no está instalado.")

In [67]:
filename = "Dataset.csv"

with open(filename, "rb") as f:
    rawdata = f.read(10000)           # Leo los primeros 10.000 bytes
    result = chardet.detect(rawdata)  # Detecto la codificación
    print(result)                     # Muestro la codificación detectada

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


#### **Carga del dataset**

In [68]:
data = pd.read_csv(filename, delimiter=';', encoding='utf-8', encoding_errors='ignore', low_memory=False)  # Cargo el archivo CSV ignorando carácteres raros.

El dataset tiene una codificación UTF-8 con un 99% de confianza, pero contiene algunos carácteres extraños que representan una cantidad despreciable de carácteres, decido ignorarlos.

### **Exploración y análisis preliminar**

Se realizará un análisis inicial del corpus, con el objetivo de comprender sus características generales. 

Se responderán las siguientes preguntas:
1. ¿Cuántos registros contiene el corpus? ¿Hay nulos?
2. ¿Cuántas palabras totales hay en los comentarios del corpus?
3. ¿Cuál el número promedio de palabras en cada comentario?

#### **<span style="color: #0098cd;">1. ¿Cuántos registros contiene el corpus? ¿Hay nulos?</span>**

In [69]:
regTotal = data.shape[0]        # Obtengo el número de registros del dataset
#regTotal = len(data)           # Otra opción para obtener el número de registros
regNulos = data.isnull().sum()  # Obtengo el número de nulos en cada columna

print(f"Número de registros en total: {regTotal}")
print("_"*37)
print(f"Valores nulos por columna: ")
print(regNulos)
print("_"*40)

Número de registros en total: 574915
_____________________________________
Valores nulos por columna: 
MEDIO                        0
SOPORTE                     34
URL                        148
TIPO DE MENSAJE            152
CONTENIDO A ANALIZAR       152
INTENSIDAD                 273
TIPO DE ODIO            562619
TONO HUMORISTICO        574769
MODIFICADOR             574562
Unnamed: 9              574912
Unnamed: 10             574912
Unnamed: 11             574913
Unnamed: 12             574913
Unnamed: 13             574913
Unnamed: 14             574913
Unnamed: 15             574913
dtype: int64
________________________________________


In [70]:
data = data.drop(columns=['Unnamed: 9', 'Unnamed: 10',         # Elimino columnas innecesarias con mayoría de nulos
                          'Unnamed: 11', 'Unnamed: 12', 
                          'Unnamed: 13', 'Unnamed: 14', 
                          'Unnamed: 15', 'TONO HUMORISTICO', 
                          'MODIFICADOR', 'TIPO DE ODIO'])


regNoNulos = data.dropna(subset=['CONTENIDO A ANALIZAR', 'INTENSIDAD']) # Elimino los registros con nulos en las columnas 'CONTENIDO A ANALIZAR' e 'INTENSIDAD'
print(f"Registros tras filtrar los nulos: {len(regNoNulos)}")
print("_"*40)

regNulos = data.isnull().sum()              # Compruebo el número de nulos en cada columna de nuevo
print(f"Valores nulos por columna: ")
print(regNulos)

Registros tras filtrar los nulos: 574642
________________________________________
Valores nulos por columna: 
MEDIO                     0
SOPORTE                  34
URL                     148
TIPO DE MENSAJE         152
CONTENIDO A ANALIZAR    152
INTENSIDAD              273
dtype: int64


Elimino las columnas con mayoría de nulos ya que no aportarán información relevante y dado que el análisis se basa en el contenido de los mensajes y su "intensidad" de odio, elimino también los registros donde faltan estos valores.

#### **<span style="color: #0098cd;">2. ¿Cuántas palabras totales hay en los comentarios del corpus?</span>**

In [71]:
# Selecciono una muestra de 15.000 registros aleatorios para pruebas más rápidas
lines_number = 15000                                      # Para cargar solo un número limitado de líneas
data_sample = data.sample(lines_number, random_state=7)   # Tomo una muestra para pruebas más rápidas

print(f"Registros en la muestra: {len(data_sample)}")     # Verifico que se han cogido los 20000 registros.
print("-"*30)

data_sample = data_sample.dropna()                        # Elimino todos los nulos de la muestra

regNulos = data_sample.isnull().sum()                     # Compruebo el número de nulos en la muestra
print(f"Valores nulos por columna en la muestra: ")
print(regNulos)

Registros en la muestra: 15000
------------------------------
Valores nulos por columna en la muestra: 
MEDIO                   0
SOPORTE                 0
URL                     0
TIPO DE MENSAJE         0
CONTENIDO A ANALIZAR    0
INTENSIDAD              0
dtype: int64


In [72]:
# Función para contar las palabras sin signos de puntuación
#----------------------------------------------------------
# is_alpha verifica que sea alfabético, es decir, que no sean signos de puntuación como -> . , ; : ! ? etc.
# is_punct verifica que es un signo de puntuación
# is_space verifica que es un espacio
# Por tanto, si el token es alfabético y no es un signo de puntuación ni un espacio, lo considero una palabra.
def CuentaPalab(mensaje):
    doc = nlp(mensaje)             # Proceso el mensaje con el modelo de SpaCy
    palabras = [token.text for token in doc if token.is_alpha and not token.is_punct and not token.is_space]
    return len(palabras)

# Ejemplo de uso de la función CuentaPalab
ejemplo = "Hola,  ¿cómo estás?   ¿Todo     bien? ¿Todo correcto?."
print("Conteo con SpaCy:", CuentaPalab(ejemplo)) # Debería dar 7

Conteo con SpaCy: 7


In [73]:
# Cuento las palabras teniendo en cuenta los signos de puntuación
ComentMuestra = data_sample[data_sample["TIPO DE MENSAJE"] == "COMENTARIO"]               # Cojo solo los que cuyo TIPO DE MENSAJE es COMENTARIO
palabTotal = ComentMuestra["CONTENIDO A ANALIZAR"].apply(lambda x: len(str(x).split()))   # Cuento el número total de palabras en los comentarios
SumapalabTotal = palabTotal.sum()
print(f"Palabras en total (contando todo) en comentarios: {SumapalabTotal}")
print("-"*51)

# Cuento las palabras sin tener en cuenta los signos de puntuación ni espacios
AlfTotal = ComentMuestra["CONTENIDO A ANALIZAR"].apply(CuentaPalab).sum()                 # Aplico la función a cada comentario y voy sumando
print(f"Palabras en total (solo alfabéticas) en los comentarios de la muestra: {AlfTotal}")
print("-"*77)


Palabras en total (contando todo) en comentarios: 395990
---------------------------------------------------
Palabras en total (solo alfabéticas) en los comentarios de la muestra: 381987
-----------------------------------------------------------------------------


Debido a los tiempos de procesamiento y a la carga computacional, para esta pregunta y para las posteriores, he seleccionado una muestra de 20.000 registros añadiendo un estado fijo para asegurar que siempre escoja los mismos registros en posteriores ejecuciones y así no distorsionar los resultados.

He considerado que los signos de puntuación no son palabras.

#### **<span style="color: #0098cd;">3. ¿Cuál el número promedio de palabras en cada comentario?</span>**

In [74]:
PalabrasPorComentario = ComentMuestra["CONTENIDO A ANALIZAR"].apply(CuentaPalab) 

AlfProm = PalabrasPorComentario.mean()           # Calculo el promedio
print(f"Promedio de palabras alfabéticas por comentario de la muestra: {AlfProm:.2f}")

Promedio de palabras alfabéticas por comentario de la muestra: 44.29


### **Análisis de entidades y características del texto**

A continuación, se aplicarán técnicas de Procesamiento de Lenguaje Natural (NLP) para extraer información significativa del corpus. Se identificará la presencia de entidades nombradas (NER), la distribución de palabras según género y número, la lematización del texto y el análisis de tendencias lingüísticas en los mensajes con y sin odio.

Para ello consideraré 2 grupos de comentarios (odio y no odio) en función de la INTENSIDAD con que estén etiquetados.

Se responderán las siguientes preguntas:

4. ¿Cuál es el número promedio de palabras en los comentarios con y sin odio?
5. ¿Cuál es el número promedio de oraciones en los comentarios con y sin odio?
6. ¿Cuál es el porcentaje de comentarios que contienen entidades NER en mensajes con y sin odio?
7. ¿Cuál es el porcentaje de comentarios que contienen entidades NER de tipo PERSON en cada grupo?
8. ¿Cuál es el porcentaje de palabras en cada combinación posible de género y número en cada grupo?
9. ¿Cuántas entidades de cada tipo posible se reconocen en cada uno de los grupos?
10. ¿Cuáles son los 10 lemas más repetidos en los comentarios de cada grupo?

#### **<span style="color: #0098cd;">4. ¿Cuál es el número promedio de palabras en los comentarios con y sin odio?</span>**

In [75]:
# Los valores de INTENSIDAD están almacenados como strings o float en el dataset, así que los convierto a enteros.
data_sample['INTENSIDAD'] = data_sample['INTENSIDAD'].astype(float).astype(int) # Convierto 'INTENSIDAD' primero a float y luego a int

# Separo los comentarios en 2 grupos: los que tienen odio y los que no
ComentNoOdio = data_sample[data_sample["INTENSIDAD"] == 0]  # Selecciono solo los comentarios sin odio (INTENSIDAD == 0)
ComentOdio = data_sample[data_sample["INTENSIDAD"] > 0]     # Selecciono solo los comentarios con odio (INTENSIDAD > 0)

# Calculo el promedio de palabras que hay en los comentarios considerados sin odio
PalabTotalNoOdio = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(CuentaPalab) 
PalabPromNoOdio = PalabTotalNoOdio.mean()
print(f"Promedio de palabras alfabéticas en comentarios sin odio: {PalabPromNoOdio:.2f}")

# Calculo el promedio de palabras que hay en los comentarios considerados con odio
PalabTotalOdio = ComentOdio["CONTENIDO A ANALIZAR"].apply(CuentaPalab)
PalabPromOdio = PalabTotalOdio.mean()
print(f"Promedio de palabras alfabéticas en comentarios con odio: {PalabPromOdio:.2f}")

Promedio de palabras alfabéticas en comentarios sin odio: 107.51
Promedio de palabras alfabéticas en comentarios con odio: 15.74


#### **<span style="color: #0098cd;">5. ¿Cuál es el número promedio de oraciones en los comentarios con y sin odio?</span>**

In [76]:
# Función para contar las oraciones
#----------------------------------
def CuentaOraciones(mensaje):
    doc = nlp(mensaje)             # Proceso el mensaje con el modelo de SpaCy
    oraciones = list(doc.sents)    # Divido el texto en oraciones usando la segmentación de Spacy
    return len(oraciones)


# Ejemplo de uso de la función CuentaOraciones
ejemplo = "Hola. ¿Cómo estás? Todo bien. Espero que tengas un buen día."
print("Conteo con SpaCy:", CuentaOraciones(ejemplo)) # Debería dar 4

Conteo con SpaCy: 4


In [77]:
#Calculo el promedio de oraciones que hay en los comentarios considerados sin odio
OracionTotalNoOdio = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(CuentaOraciones)
OracionPromNoOdio = OracionTotalNoOdio.mean()
print(f"Promedio de oraciones en comentarios sin odio: {OracionPromNoOdio:.2f}")

#Calculo el promedio de oraciones que hay en los comentarios considerados con odio
OracionTotalOdio = ComentOdio["CONTENIDO A ANALIZAR"].apply(CuentaOraciones)
OracionPromOdio = OracionTotalOdio.mean()
print(f"Promedio de oraciones en comentarios con odio: {OracionPromOdio:.2f}")

Promedio de oraciones en comentarios sin odio: 4.00
Promedio de oraciones en comentarios con odio: 1.49


#### **<span style="color: #0098cd;">6. ¿Cuál es el porcentaje de comentarios que contienen entidades NER en mensajes con y sin odio?</span>**

NER (Named Entity Recognition, Reconocimiento de Entidades Nombradas) es una técnica de NLP utilizada para identificar y clasificar palabras o frases específicas dentro de un texto. Estas palabras representan entidades con significado propio, como nombres de personas, lugares, organizaciones, fechas, cantidades, etc..

Por ejemplo: 
**"Elon Musk es el CEO de Tesla, una empresa con sede en California, y su fortuna supera los 200 mil millones de dólares."**

| **Entidad**                     | **Categoría (NER)**             |
|---------------------------------|--------------------------------|
| Elon Musk                       | **PERSON** (Persona)           |
| Tesla                           | **ORG** (Organización)         |
| California                      | **GPE** (Ubicación geopolítica) |
| 200 mil millones de dólares     | **MONEY** (Dinero)             |


In [78]:
# Ejemplo de funcionamiento
ejemplo = "Lionel Messi juega en el Inter de Miami, un club de fútbol en Estados Unidos."

doc = nlp(ejemplo)          # Proceso el mensaje con el modelo de SpaCy
for entidad in doc.ents:    # Itero sobre las entidades detectadas
    print(f"Entidad: {entidad.text} - Tipo: {entidad.label_}")

Entidad: Lionel Messi - Tipo: PER
Entidad: Inter de Miami - Tipo: ORG
Entidad: en Estados Unidos - Tipo: LOC


In [79]:
# Función para contar las entidades
#----------------------------------
def TieneEntidades(mensaje):
    doc = nlp(mensaje)         # Proceso el mensaje con el modelo de SpaCy
    return len(doc.ents) > 0   # True si al menos hay una entidad

In [80]:
# Creo copias para evitar modificar el dataframe original
ComentNoOdio = ComentNoOdio.copy()
ComentOdio = ComentOdio.copy()

# Calculo el porcentaje de entidades NER en comentarios sin odio
ComentNoOdio['tiene_entidades'] = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(TieneEntidades)    # Aplico la función a cada comentario
PorcEntidadesNoOdio = 100 * ComentNoOdio['tiene_entidades'].mean()                              # Calculo el porcentaje de comentarios sin odio y con entidades NER
print(f"Porcentaje de comentarios sin odio y con entidades NER: {PorcEntidadesNoOdio:.2f}%")

# Calculo el porcentaje de entidades NER en comentarios con odio
ComentOdio['tiene_entidades'] = ComentOdio["CONTENIDO A ANALIZAR"].apply(TieneEntidades)         # Aplico la función a cada comentario
PorcEntidadesOdio = 100 * ComentOdio['tiene_entidades'].mean()                                   # Calculo el porcentaje de comentarios con odio y con entidades NER
print(f"Porcentaje de comentarios con odio y con entidades NER: {PorcEntidadesOdio:.2f}%")

Porcentaje de comentarios sin odio y con entidades NER: 59.67%
Porcentaje de comentarios con odio y con entidades NER: 38.54%


#### **<span style="color: #0098cd;">7. ¿Cuál es el porcentaje de comentarios que contienen entidades NER de tipo PERSON en cada grupo?</span>**

In [81]:
# Función para contar las entidades de tipo PERSON
#----------------------------------
def TienePerson(mensaje): 
    doc = nlp(mensaje)                                   # Proceso el mensaje con el modelo de SpaCy
    return any(ent.label_ == 'PER' for ent in doc.ents)  # True si al menos hay una entidad de tipo PERSON (PER)

In [82]:
# Calculo el porcentaje de entidades PERSON en comentarios sin odio
ComentNoOdio['tiene_personas'] = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(TienePerson)         # Aplico la función a cada comentario
PorcPersonNoOdio = 100 * ComentNoOdio['tiene_personas'].mean()                                   # Calculo el porcentaje de comentarios sin odio y con entidades de tipo PERSON
print(f"Porcentaje de comentarios sin odio con entidades NER de tipo PERSON: {PorcPersonNoOdio:.2f}%")


# Calculo el porcentaje de entidades PERSON en comentarios con odio
ComentOdio['tiene_personas'] = ComentOdio["CONTENIDO A ANALIZAR"].apply(TienePerson)             # Aplico la función a cada comentario
PorcPersonOdio = 100 * ComentOdio['tiene_personas'].mean()                                       # Calculo el porcentaje de comentarios con odio y con entidades de tipo PERSON
print(f"Porcentaje de comentarios con odio con entidades NER de tipo PERSON: {PorcPersonOdio:.2f}%")

Porcentaje de comentarios sin odio con entidades NER de tipo PERSON: 28.89%
Porcentaje de comentarios con odio con entidades NER de tipo PERSON: 19.75%


#### **<span style="color: #0098cd;">8. ¿Cuál es el porcentaje de palabras en cada combinación posible de género y número en cada grupo?</span>**

In [83]:
# Función para contar las palabras según su género y número
#----------------------------------------------------------
def CuentaGeneroyNumero(mensaje):
    doc = nlp(mensaje)
    conteo = {
        'ms': 0, 'mp': 0, 'fs': 0, 'fp': 0}
    for token in doc:               # Itero sobre cada token en el documento
        if token.is_alpha:  
            genero = token.morph.get('Gender')
            numero = token.morph.get('Number')
            if genero and numero:
                if 'Masc' in genero and 'Sing' in numero:   # Masculino singular
                    conteo['ms'] += 1  
                elif 'Masc' in genero and 'Plur' in numero: # Masculino plural
                    conteo['mp'] += 1  
                elif 'Fem' in genero and 'Sing' in numero:  # Femenino singular
                    conteo['fs'] += 1  
                elif 'Fem' in genero and 'Plur' in numero:  # Femenino plural
                    conteo['fp'] += 1  
    return conteo

In [84]:
# Cálculo del porcentaje para comentarios sin odio
ComentNoOdio['genero_numero'] = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(CuentaGeneroyNumero) # Aplico la función a cada comentario
GenyNumTotalNoOdio = {'ms': 0, 'fs': 0, 'mp': 0, 'fp': 0}                                       # Diccionario para el conteo total de cada categoría

for res in ComentNoOdio['genero_numero']:       # Itero sobre cada diccionario de resultados
    for key in res:                             # Itero sobre las claves ('ms', 'fs', 'mp', 'fp')
        GenyNumTotalNoOdio[key] += res[key]     # Sumo las cantidades al diccionario acumulador

PalabTotalNO = sum(GenyNumTotalNoOdio.values())                                                         # Calculo el total de palabras analizadas en los comentarios sin odio
PorcNoOdio = {key: round((value / PalabTotalNO) * 100, 2) for key, value in GenyNumTotalNoOdio.items()} # Calculo el porcentaje de cada categoría de palabras


# Cálculo del porcentaje para comentarios con odio
ComentOdio['genero_numero'] = ComentOdio["CONTENIDO A ANALIZAR"].apply(CuentaGeneroyNumero)    # Aplico la función a cada comentario
GenyNunTotalOdio = {'ms': 0, 'fs': 0, 'mp': 0, 'fp': 0}                                        # Diccionario para el conteo total de cada categoría

for res in ComentOdio['genero_numero']:         # Itero sobre cada diccionario de resultados
    for key in res:                             # Itero sobre las claves ('ms', 'fs', 'mp', 'fp')
        GenyNunTotalOdio[key] += res[key]       # Sumo las cantidades al diccionario acumulador

PalabTotalO = sum(GenyNunTotalOdio.values())                                                       # Calculo el total de palabras analizadas en los comentarios con odio
PorcOdio = {key: round((value / PalabTotalO) * 100, 2) for key, value in GenyNunTotalOdio.items()} # Calculo el porcentaje de cada categoría de palabras

print("Porcentajes por combinaciones en comentarios sin odio:", PorcNoOdio)
print("Porcentajes por combinaciones en comentarios con odio:", PorcOdio)

Porcentajes por combinaciones en comentarios sin odio: {'ms': 40.78, 'fs': 33.64, 'mp': 14.9, 'fp': 10.68}
Porcentajes por combinaciones en comentarios con odio: {'ms': 40.87, 'fs': 32.07, 'mp': 18.57, 'fp': 8.48}


#### **<span style="color: #0098cd;">9. ¿Cuántas entidades de cada tipo posible se reconocen en cada uno de los grupos?</span>**

In [85]:
# Función para contar las NER según su tipo
#------------------------------------------
def CuentaEntidadesTipo(mensaje):
    doc = nlp(mensaje)                          # Proceso el mensaje con el modelo de SpaCy
    conteo_entidades = {}                       # Diccionario para almacenar el número de entidades por tipo
    
    for ent in doc.ents:                        # Itero sobre todas las entidades detectadas
        if ent.label_ in conteo_entidades:      # Si ya existe el tipo de entidad, suma 1
            conteo_entidades[ent.label_] += 1
        else:                                   # Si es la primera vez que aparece ese tipo de entidad, la inicializo con 1
            conteo_entidades[ent.label_] = 1
    return conteo_entidades                     # Devuelvo el diccionario con el conteo de entidades

In [86]:
ComentNoOdio['entidades_tipo'] = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(CuentaEntidadesTipo) # Aplico la función a cada comentario
EntidadesTotalNoOdio = {}                       # Diccionario para almacenar el número de entidades por tipo en comentarios sin odio

# Sumo todas las entidades encontradas
for entidades in ComentNoOdio['entidades_tipo']:
    for tipo, cantidad in entidades.items():
        if tipo in EntidadesTotalNoOdio:             # Si ya existe el tipo de entidad, sumo su cantidad
            EntidadesTotalNoOdio[tipo] += cantidad
        else:                                        # Si es la primera vez que aparece, la inicializo con la cantidad
            EntidadesTotalNoOdio[tipo] = cantidad



ComentOdio['entidades_tipo'] = ComentOdio["CONTENIDO A ANALIZAR"].apply(CuentaEntidadesTipo)     # Aplico la función a cada comentario
EntidadesTotalOdio = {}                        # Diccionario para almacenar el número de entidades por tipo en comentarios con odio

# Sumo todas las entidades encontradas
for entidades in ComentOdio['entidades_tipo']:
    for tipo, cantidad in entidades.items():
        if tipo in EntidadesTotalOdio:
            EntidadesTotalOdio[tipo] += cantidad
        else:
            EntidadesTotalOdio[tipo] = cantidad

print("Tipos de entidades en total en comentarios sin odio:", EntidadesTotalNoOdio)
print("Tipos de entidades en total en comentarios con odio:", EntidadesTotalOdio)

Tipos de entidades en total en comentarios sin odio: {'LOC': 19305, 'MISC': 5333, 'PER': 17360, 'ORG': 5547}
Tipos de entidades en total en comentarios con odio: {'LOC': 36, 'PER': 71, 'MISC': 31, 'ORG': 31}


#### **<span style="color: #0098cd;">10. ¿Cuáles son los 10 lemas más repetidos en los comentarios de cada grupo?</span>**

La realicé una primera vez sin tener en cuenta stopwords para ver los resultados que obtenía. Tras su ejecución vi que había un buen número de "i" y "q" que realmente no aportan mucha información a los resultados por lo que los añadí como stopword para filtrarlas.

In [87]:
from spacy.lang.es import STOP_WORDS  # Importo la lista de stopwords del español

for word in ['q', 'i']: 
    if word not in STOP_WORDS:  # Agrego 'q' e 'i' a la lista de stopwords si no lo están ya
        STOP_WORDS.add(word)

# Función para extraer lemas
#---------------------------
def SacaLemas(mensaje):
    doc = nlp(mensaje)          # Proceso el mensaje con el modelo de SpaCy
    lemas = [token.lemma_ for token in doc if token.is_alpha and token.text not in STOP_WORDS]
    return lemas

In [ ]:
from collections import Counter  # Para el conteo

LemasNoOdio = ComentNoOdio["CONTENIDO A ANALIZAR"].apply(SacaLemas).sum() # Aplico la función a cada comentario y sumo los resultados
CuentaLemasNoOdio = Counter(LemasNoOdio)                                  # Cuento cuántas veces aparece cada lema
LemasRepetidosNoOdio = CuentaLemasNoOdio.most_common(10)                  # Obtengo los 10 lemas más repetidos


LemasOdio = ComentOdio["CONTENIDO A ANALIZAR"].apply(SacaLemas).sum()     # Aplico la función a cada comentario y sumo los resultados
CuentaLemasOdio = Counter(LemasOdio)                                      # Cuento cuántas veces aparece cada lema
LemasRepetidosOdio = CuentaLemasOdio.most_common(10)                      # Obtengo los 10 lemas más repetidos


print("10 lemas más repetidos en comentarios sin odio:")
for lema, frecuencia in LemasRepetidosNoOdio:
    print(f"{lema}: {frecuencia}")

print("\n10 lemas más repetidos en comentarios con odio:")
for lema, frecuencia in LemasRepetidosOdio:
    print(f"{lema}: {frecuencia}")

100 lemas más repetidos en comentarios sin odio:
año: 4403
persona: 2365
gobierno: 2203
país: 2006
caso: 1964
españa: 1643
madrid: 1490
ver: 1450
millón: 1406
llegar: 1352

100 lemas más repetidos en comentarios con odio:
puta: 26
mierda: 26
hijo: 15
gobierno: 14
gente: 12
asco: 12
gentuza: 12
panfleto: 11
mentiroso: 10
ir: 10


### **Análisis final**

#### **<span style="color: #0098cd;">Con los resultados obtenidos, ¿es posible identificar tendencias generales para determinar si un mensaje contiene odio?</span>**

**Recopilando los resultados más relevantes obtenidos:**

- Promedio de palabras alfabéticas por comentario de la muestra: 44.29
- Promedio de palabras alfabéticas en comentarios sin odio: 107.51
- Promedio de palabras alfabéticas en comentarios con odio: 15.74
- Promedio de oraciones en comentarios sin odio: 4.00
- Promedio de oraciones en comentarios con odio: 1.49
- Porcentaje de comentarios sin odio con entidades NER: 59.67%
- Porcentaje de comentarios con odio con entidades NER: 38.54%
- Porcentaje de comentarios sin odio con entidades NER de tipo PERSON: 28.89%
- Porcentaje de comentarios con odio con entidades NER de tipo PERSON: 19.75%
- Porcentajes por combinaciones en comentarios sin odio: {'ms': 40.78, 'fs': 33.64, 'mp': 14.9, 'fp': 10.68}
- Porcentajes por combinaciones en comentarios con odio: {'ms': 40.87, 'fs': 32.07, 'mp': 18.57, 'fp': 8.48}
- Tipos de entidades en total en comentarios sin odio: {'LOC': 19305, 'MISC': 5333, 'PER': 17360, 'ORG': 5547}
- Tipos de entidades en total en comentarios con odio: {'LOC': 36, 'PER': 71, 'MISC': 31, 'ORG': 31}
- 5 lemas más repetidos en comentarios sin odio:
  - año: 4403
  - persona: 2365
  - gobierno: 2203
  - país: 2006
  - caso: 1964
- 5 lemas más repetidos en comentarios con odio:
  - puta: 26
  - mierda: 26
  - hijo: 15
  - gobierno: 14
  - gente: 12

Con estos resultados se puede ver que hay ciertas tendencias generales y diferencias significativas entre los mensajes con y sin odio:

- Los comentarios sin odio tienen en promedio 107.51 palabras, mientras que los comentarios con odio solo tienen 15.74 palabras. Esto indica que los mensajes con odio suelen ser más cortos y directos, mientras que los mensajes sin odio son más generales y explicativos.

- Los comentarios sin odio contienen en promedio 4 oraciones, mientras que los comentarios con odio solo 1.49. Esto confirma que los mensajes con odio suelen ser más concisos, lo que puede deberse a su carácter impulsivo o agresivo.

- El 59.67% de los comentarios sin odio contienen entidades NER, mientras que en los comentarios con odio este porcentaje baja al 38.54%. Además, la presencia de entidades PERSON es menor en comentarios con odio (19.75%) que en comentarios sin odio (28.89%). Esto refleja que los mensajes sin odio tienden a mencionar más nombres de personas y lugares, mientras que los mensajes con odio pueden centrarse más en insultos genéricos o expresiones sin referencias concretas.

- En ambos tipos de comentarios, la mayoría de palabras son masculinas singulares (ms), pero hay más palabras en plural en los comentarios con odio (mp: 18.57% vs. mp: 14.9% en sin odio).
Esto puede indicar que los mensajes con odio tienden a generalizar más en sus afirmaciones, en lugar de dirigirse a individuos específicos.

- En comentarios con odio, las palabras más comunes son insultos y términos despectivos. Por lo que el lenguaje en los comentarios con odio se centra en agresiones verbales, mientras que en los comentarios sin odio hay una mayor diversidad de temas.